# Machine Leanring Engineer Nanodegree
------
## Capstone Project:  Flight Delay Prediction
---
This MLND Captose Project for . In this we will use DOT's dataset. 

## Import All The Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib as mpl
import seaborn as sns
from datetime import datetime as dt  
from utils.MlUtils import MlUtils ,convert_time ,concat_date_time ,concat_date_time_ARRIVAL, concat_date_time_TaxiOut
from matplotlib.gridspec import GridSpec
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler

### Load Dataset 
- Loading the dataset for Flight delay Prediction. Dataset in in CSV format.

In [2]:
# Instance of Utility class
ut=MlUtils()

In [3]:
dfFlight = ut.load_csv('././data/flights.csv', None)
dfFlight.shape

(5819079, 31)

In [4]:
dfFlight.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


- As Dataset is high in volume(~5819079). So for computation, it required high computation power. Because of this limitation, i am going to use only one month data for this project.


In [5]:
dfFlight=dfFlight[dfFlight['MONTH']==1]
dfFlight.shape

(469968, 31)

In [6]:
# dfFlight=dfFlight.iloc[:69968,:]
dfFlight.shape

(469968, 31)

- Data type of all feature given in flight.csv data. It will help in understading of data for future analysis.

In [7]:
dfFlight.dtypes

YEAR                     int64
MONTH                    int64
DAY                      int64
DAY_OF_WEEK              int64
AIRLINE                 object
FLIGHT_NUMBER            int64
TAIL_NUMBER             object
ORIGIN_AIRPORT          object
DESTINATION_AIRPORT     object
SCHEDULED_DEPARTURE      int64
DEPARTURE_TIME         float64
DEPARTURE_DELAY        float64
TAXI_OUT               float64
WHEELS_OFF             float64
SCHEDULED_TIME         float64
ELAPSED_TIME           float64
AIR_TIME               float64
DISTANCE                 int64
WHEELS_ON              float64
TAXI_IN                float64
SCHEDULED_ARRIVAL        int64
ARRIVAL_TIME           float64
ARRIVAL_DELAY          float64
DIVERTED                 int64
CANCELLED                int64
CANCELLATION_REASON     object
AIR_SYSTEM_DELAY       float64
SECURITY_DELAY         float64
AIRLINE_DELAY          float64
LATE_AIRCRAFT_DELAY    float64
WEATHER_DELAY          float64
dtype: object

- In the below section, we will check missing values of features and will calculate the % of missing values.
- So in case, high degree of missing percentage of values, we will remove that particular feature.
- `Showing Percentage of missing values per feature`

In [6]:

ut.sample_count=dfFlight.shape[0]
ut.missingPercent(dfFlight)

                     COUNT_OF_MISSING  COUNT_OF_MISSING_%
FEATURE                                                  
YEAR                                0            0.000000
MONTH                               0            0.000000
DAY                                 0            0.000000
DAY_OF_WEEK                         0            0.000000
AIRLINE                             0            0.000000
FLIGHT_NUMBER                       0            0.000000
TAIL_NUMBER                      2782            0.591955
ORIGIN_AIRPORT                      0            0.000000
DESTINATION_AIRPORT                 0            0.000000
SCHEDULED_DEPARTURE                 0            0.000000
DEPARTURE_TIME                  11657            2.480382
DEPARTURE_DELAY                 11657            2.480382
TAXI_OUT                        11876            2.526981
WHEELS_OFF                      11876            2.526981
SCHEDULED_TIME                      0            0.000000
ELAPSED_TIME  

#### Removing the feature with high mising percentage and unqique identifier 
- `FLIGHT_NUMBER`: This a unique identifier for the flight. 
- `TAIL_NUMBER` : This is also a unique identifier number for flight. It's callled aircraft registation number.
- `CANCELLATION_REASON`, `AIR_SYSTEM_DELAY`,  `SECURITY_DELAY`, `AIRLINE_DELAY`, `LATE_AIRCRAFT_DELAY`, `WEATHER_DELAY`:  As this is has high degree of missing values. if will we try ot fill it, it will add high volume of redudent data. So, instead of filling it. we will remove them from our dataframe. 

In [7]:
print(dfFlight.shape) 
variables_to_remove = ['FLIGHT_NUMBER', 'TAIL_NUMBER', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY',
                       'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
dfFlight.drop(variables_to_remove, axis=1, inplace=True)
print(dfFlight.shape)

(469968, 31)
(469968, 23)


In [8]:
dfFlight.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,...,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED
0,2015,1,1,4,AS,ANC,SEA,5,2354.0,-11.0,...,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0
1,2015,1,1,4,AA,LAX,PBI,10,2.0,-8.0,...,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0
2,2015,1,1,4,US,SFO,CLT,20,18.0,-2.0,...,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0
3,2015,1,1,4,AA,LAX,MIA,20,15.0,-5.0,...,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0
4,2015,1,1,4,AS,SEA,ANC,25,24.0,-1.0,...,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0


####  Drived Features
- I will drive a feature `FLIGHT_DATE`, which will be combination of our dataframe's three feature `YEAR`,`MONTH`,`DAY`.It will drive flight journy date.
- And later on we will remove these three feature from our dataset. 

In [9]:
t2=dt.now()
dfFlight['FLIGHT_DATE'] = pd.to_datetime(dfFlight[['YEAR', 'MONTH', 'DAY']])
print(dt.now() -t2)

0:00:00.334002


In [10]:
variables_to_remove=['YEAR', 'MONTH', 'DAY']
dfFlight.drop(variables_to_remove, axis=1, inplace=True)
print(dfFlight.shape)

(469968, 21)


- As `ARRIVAL_DELAY` is flight delay on arrival airport. In dataset, it a numeric value. But as per our problem we need to convert it to classification problem. We will drive another feature `FLIGHT_DELAY` with `YES` or `NO`

In [11]:
t2=dt.now()
dfFlight['FLIGHT_DELAY'] = dfFlight['ARRIVAL_DELAY'].map( lambda x : 'YES' if x>0 else 'NO')
print(dfFlight.shape)
print(dt.now() -t2)

(469968, 22)
0:00:00.126929


In [12]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,...,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,FLIGHT_DATE,FLIGHT_DELAY
0,4,AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,...,1448,404.0,4.0,430,408.0,-22.0,0,0,2015-01-01,NO
1,4,AA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,...,2330,737.0,4.0,750,741.0,-9.0,0,0,2015-01-01,NO
2,4,US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,...,2296,800.0,11.0,806,811.0,5.0,0,0,2015-01-01,YES
3,4,AA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,...,2342,748.0,8.0,805,756.0,-9.0,0,0,2015-01-01,NO
4,4,AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,...,1448,254.0,5.0,320,259.0,-21.0,0,0,2015-01-01,NO


In [13]:
t4=dt.now()
dfFlight['SCHEDULED_DEPARTURE'] = dfFlight.apply(concat_date_time, axis=1)
print(dt.now() -t4)

0:01:09.760792


In [14]:
t5=dt.now()
dfFlight['DEPARTURE_TIME'] = dfFlight['DEPARTURE_TIME'].map(convert_time)
print(dt.now() -t5)

0:00:39.156418


In [15]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,...,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,FLIGHT_DATE,FLIGHT_DELAY
0,4,AS,ANC,SEA,2015-01-01 00:05:00,23:54:00,-11.0,21.0,15.0,205.0,...,1448,404.0,4.0,430,408.0,-22.0,0,0,2015-01-01,NO
1,4,AA,LAX,PBI,2015-01-01 00:10:00,00:02:00,-8.0,12.0,14.0,280.0,...,2330,737.0,4.0,750,741.0,-9.0,0,0,2015-01-01,NO
2,4,US,SFO,CLT,2015-01-01 00:20:00,00:18:00,-2.0,16.0,34.0,286.0,...,2296,800.0,11.0,806,811.0,5.0,0,0,2015-01-01,YES
3,4,AA,LAX,MIA,2015-01-01 00:20:00,00:15:00,-5.0,15.0,30.0,285.0,...,2342,748.0,8.0,805,756.0,-9.0,0,0,2015-01-01,NO
4,4,AS,SEA,ANC,2015-01-01 00:25:00,00:24:00,-1.0,11.0,35.0,235.0,...,1448,254.0,5.0,320,259.0,-21.0,0,0,2015-01-01,NO


In [16]:
t6=dt.now()
dfFlight['SCHEDULED_ARRIVAL'] =  dfFlight.apply(concat_date_time_ARRIVAL, axis=1)
print(dfFlight.columns)
print(dfFlight.shape)
print(dt.now() -t6)

Index(['DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'FLIGHT_DATE',
       'FLIGHT_DELAY'],
      dtype='object')
(469968, 22)
0:01:13.051605


In [20]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,...,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,FLIGHT_DATE,FLIGHT_DELAY,TOTAL_JOURNERY_TIME
0,4,AS,ANC,SEA,2015-01-01 00:05:00,23:54:00,-11.0,21.0,15.0,205.0,...,404.0,4.0,2015-01-01 04:30:00,408.0,-22.0,0,0,2015-01-01,NO,04:25:00
1,4,AA,LAX,PBI,2015-01-01 00:10:00,00:02:00,-8.0,12.0,14.0,280.0,...,737.0,4.0,2015-01-01 07:50:00,741.0,-9.0,0,0,2015-01-01,NO,07:40:00
2,4,US,SFO,CLT,2015-01-01 00:20:00,00:18:00,-2.0,16.0,34.0,286.0,...,800.0,11.0,2015-01-01 08:06:00,811.0,5.0,0,0,2015-01-01,YES,07:46:00
3,4,AA,LAX,MIA,2015-01-01 00:20:00,00:15:00,-5.0,15.0,30.0,285.0,...,748.0,8.0,2015-01-01 08:05:00,756.0,-9.0,0,0,2015-01-01,NO,07:45:00
4,4,AS,SEA,ANC,2015-01-01 00:25:00,00:24:00,-1.0,11.0,35.0,235.0,...,254.0,5.0,2015-01-01 03:20:00,259.0,-21.0,0,0,2015-01-01,NO,02:55:00


In [21]:
# Total planned Journery Time 
dfFlight['TOTAL_JOURNERY_TIME']=dfFlight['SCHEDULED_ARRIVAL']- dfFlight['SCHEDULED_DEPARTURE']

In [22]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,...,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,FLIGHT_DATE,FLIGHT_DELAY,TOTAL_JOURNERY_TIME
0,4,AS,ANC,SEA,2015-01-01 00:05:00,23:54:00,-11.0,21.0,15.0,205.0,...,404.0,4.0,2015-01-01 04:30:00,408.0,-22.0,0,0,2015-01-01,NO,04:25:00
1,4,AA,LAX,PBI,2015-01-01 00:10:00,00:02:00,-8.0,12.0,14.0,280.0,...,737.0,4.0,2015-01-01 07:50:00,741.0,-9.0,0,0,2015-01-01,NO,07:40:00
2,4,US,SFO,CLT,2015-01-01 00:20:00,00:18:00,-2.0,16.0,34.0,286.0,...,800.0,11.0,2015-01-01 08:06:00,811.0,5.0,0,0,2015-01-01,YES,07:46:00
3,4,AA,LAX,MIA,2015-01-01 00:20:00,00:15:00,-5.0,15.0,30.0,285.0,...,748.0,8.0,2015-01-01 08:05:00,756.0,-9.0,0,0,2015-01-01,NO,07:45:00
4,4,AS,SEA,ANC,2015-01-01 00:25:00,00:24:00,-1.0,11.0,35.0,235.0,...,254.0,5.0,2015-01-01 03:20:00,259.0,-21.0,0,0,2015-01-01,NO,02:55:00


### Percentage of missing values per columns

In [23]:
ut.missingPercent(dfFlight)

                     COUNT_OF_MISSING  COUNT_OF_MISSING_%
FEATURE                                                  
DAY_OF_WEEK                         0            0.000000
AIRLINE                             0            0.000000
ORIGIN_AIRPORT                      0            0.000000
DESTINATION_AIRPORT                 0            0.000000
SCHEDULED_DEPARTURE                 0            0.000000
DEPARTURE_TIME                  11691            2.487616
DEPARTURE_DELAY                 11657            2.480382
TAXI_OUT                        11876            2.526981
WHEELS_OFF                      11876            2.526981
SCHEDULED_TIME                      0            0.000000
ELAPSED_TIME                    12955            2.756571
AIR_TIME                        12955            2.756571
DISTANCE                            0            0.000000
WHEELS_ON                       12271            2.611029
TAXI_IN                         12271            2.611029
SCHEDULED_ARRI

### Removing Features which are not more relevant to our model 

- `ARRIVAL_DELAY` : Drived `FLIGHT_DELAY`
- `YEAR`, `MONTH`, `DAY` : We drived anohter feature by using them, So i am going to remove them.
- `FLIGHT_NUMBER` : It's flight number like other vehical number. it
- `TAIL_NUMBER` : It's unique identifier for Aircraft. It's called Aircart registration number. 
- `CANCELLATION_REASON` , `AIR_SYSTEM_DELAY` , `SECURITY_DELAY`, `AIRLINE_DELAY`, `LATE_AIRCRAFT_DELAY`, `WEATHER_DELAY` :  Aprox 82% value are missing for these features. 
- `AIR_TIME` : This will be populated after landing. So it's not important feature for our model.
- `FLIGHT_DATE` : Not relevent 
- `ARRIVAL_TIME` ,`WHEELS_ON`, `TAXI_IN` , `CANCELLED` : This will be populated after landing on airport.

- `WHEELS_ON` : The time point that the aircraft's wheels touch on the ground
- `TAXI_IN` : The time duration elapsed between wheels-on and gate arrival at the destination airport
- `ARRIVAL_TIME` : WHEELS_ON+TAXI_IN

- `ELAPSED_TIME` : AIR_TIME+TAXI_IN+TAXI_OUT
 

In [24]:
# Removing because  We have drive the Flight_Delay  from  ARRIVAL_DELAY. So Arrival_Delay is no relevent more.
variables_to_remove=['ARRIVAL_DELAY','ELAPSED_TIME' , 'AIR_TIME', 'SCHEDULED_DEPARTURE','SCHEDULED_ARRIVAL',
                     'FLIGHT_DATE','ARRIVAL_TIME' , 'WHEELS_ON', 'TAXI_IN','CANCELLED'] 
dfFlight.drop(variables_to_remove, axis=1, inplace=True)
print(dfFlight.shape)

(469968, 13)


In [25]:
dfFlight.columns

Index(['DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'SCHEDULED_TIME', 'DISTANCE', 'DIVERTED', 'FLIGHT_DELAY',
       'TOTAL_JOURNERY_TIME'],
      dtype='object')

In [26]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,DIVERTED,FLIGHT_DELAY,TOTAL_JOURNERY_TIME
0,4,AS,ANC,SEA,23:54:00,-11.0,21.0,15.0,205.0,1448,0,NO,04:25:00
1,4,AA,LAX,PBI,00:02:00,-8.0,12.0,14.0,280.0,2330,0,NO,07:40:00
2,4,US,SFO,CLT,00:18:00,-2.0,16.0,34.0,286.0,2296,0,YES,07:46:00
3,4,AA,LAX,MIA,00:15:00,-5.0,15.0,30.0,285.0,2342,0,NO,07:45:00
4,4,AS,SEA,ANC,00:24:00,-1.0,11.0,35.0,235.0,1448,0,NO,02:55:00


In [27]:
# Removing because  We have drive the Flight_Delay  from  ARRIVAL_DELAY. So Arrival_Delay is no relevent more.
variables_to_remove=['DEPARTURE_TIME','SCHEDULED_TIME','WHEELS_OFF'] 
dfFlight.drop(variables_to_remove, axis=1, inplace=True)
print(dfFlight.shape)

(469968, 10)


In [28]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,DISTANCE,DIVERTED,FLIGHT_DELAY,TOTAL_JOURNERY_TIME
0,4,AS,ANC,SEA,-11.0,21.0,1448,0,NO,04:25:00
1,4,AA,LAX,PBI,-8.0,12.0,2330,0,NO,07:40:00
2,4,US,SFO,CLT,-2.0,16.0,2296,0,YES,07:46:00
3,4,AA,LAX,MIA,-5.0,15.0,2342,0,NO,07:45:00
4,4,AS,SEA,ANC,-1.0,11.0,1448,0,NO,02:55:00


### Data verification realated to Imbalance or Balance
----

In [29]:
# Calculate Count of classes per class
dfDelayClass=dfFlight['FLIGHT_DELAY'].T.value_counts().reset_index() 
dfDelayClass.columns=["FLIGHT_DELAY_CLASS","COUNT_PER_CLASS"]
dfDelayClass

,FLIGHT_DELAY_CLASS,COUNT_PER_CLASS
0,NO,286858
1,YES,183110


In [30]:
dfDelayClass['%_PER_CLASS']=dfDelayClass['COUNT_PER_CLASS'].map(ut.Missing_Count)
# dMissing_CountfDelayClass
dfDelayClass

,FLIGHT_DELAY_CLASS,COUNT_PER_CLASS,%_PER_CLASS
0,NO,286858,61.037773
1,YES,183110,38.962227


In [31]:
ut.missingPercent(dfFlight=dfFlight)

                     COUNT_OF_MISSING  COUNT_OF_MISSING_%
FEATURE                                                  
DAY_OF_WEEK                         0            0.000000
AIRLINE                             0            0.000000
ORIGIN_AIRPORT                      0            0.000000
DESTINATION_AIRPORT                 0            0.000000
DEPARTURE_DELAY                 11657            2.480382
TAXI_OUT                        11876            2.526981
DISTANCE                            0            0.000000
DIVERTED                            0            0.000000
FLIGHT_DELAY                        0            0.000000
TOTAL_JOURNERY_TIME                 0            0.000000
0:00:00.159194


In [32]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,DISTANCE,DIVERTED,FLIGHT_DELAY,TOTAL_JOURNERY_TIME
0,4,AS,ANC,SEA,-11.0,21.0,1448,0,NO,04:25:00
1,4,AA,LAX,PBI,-8.0,12.0,2330,0,NO,07:40:00
2,4,US,SFO,CLT,-2.0,16.0,2296,0,YES,07:46:00
3,4,AA,LAX,MIA,-5.0,15.0,2342,0,NO,07:45:00
4,4,AS,SEA,ANC,-1.0,11.0,1448,0,NO,02:55:00


### Airline Details 
---


In [33]:
# Converting Day_of_week in Catogical Feature
week_Days={1:'ONE',2:'TWO',3:'THREE',4:'FOUR',5:'FIVE',6:'SIX',7:'SEVEN'}
delay_binary={'YES':0,'NO':1}

In [34]:
dfFlight['DAY_OF_WEEK']=dfFlight['DAY_OF_WEEK'].map(week_Days)
dfFlight['FLIGHT_DELAY']=dfFlight['FLIGHT_DELAY'].map(delay_binary)
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,DISTANCE,DIVERTED,FLIGHT_DELAY,TOTAL_JOURNERY_TIME
0,FOUR,AS,ANC,SEA,-11.0,21.0,1448,0,1,04:25:00
1,FOUR,AA,LAX,PBI,-8.0,12.0,2330,0,1,07:40:00
2,FOUR,US,SFO,CLT,-2.0,16.0,2296,0,0,07:46:00
3,FOUR,AA,LAX,MIA,-5.0,15.0,2342,0,1,07:45:00
4,FOUR,AS,SEA,ANC,-1.0,11.0,1448,0,1,02:55:00


#### Feature Engineering 
---
`We will convert all the time in minutes in Decimal`

- `ORIGIN_AIRPORT` `DESTINATION_AIRPORT` : `LAG_INFO`


In [35]:
dfFlight['LAG_INFO']= dfFlight['ORIGIN_AIRPORT']+ '--' +dfFlight['DESTINATION_AIRPORT']
dfFlight.shape

(469968, 11)

In [36]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,DISTANCE,DIVERTED,FLIGHT_DELAY,TOTAL_JOURNERY_TIME,LAG_INFO
0,FOUR,AS,ANC,SEA,-11.0,21.0,1448,0,1,04:25:00,ANC--SEA
1,FOUR,AA,LAX,PBI,-8.0,12.0,2330,0,1,07:40:00,LAX--PBI
2,FOUR,US,SFO,CLT,-2.0,16.0,2296,0,0,07:46:00,SFO--CLT
3,FOUR,AA,LAX,MIA,-5.0,15.0,2342,0,1,07:45:00,LAX--MIA
4,FOUR,AS,SEA,ANC,-1.0,11.0,1448,0,1,02:55:00,SEA--ANC


In [37]:
# Removing because  We have drive the Flight_Delay  from  ARRIVAL_DELAY. So Arrival_Delay is no relevent more.
variables_to_remove=['DESTINATION_AIRPORT','ORIGIN_AIRPORT']
dfFlight.drop(variables_to_remove, axis=1, inplace=True)
print(dfFlight.shape)

(469968, 9)


In [38]:
print(dfFlight['LAG_INFO'].nunique())
dfFlight.head()

4170


,DAY_OF_WEEK,AIRLINE,DEPARTURE_DELAY,TAXI_OUT,DISTANCE,DIVERTED,FLIGHT_DELAY,TOTAL_JOURNERY_TIME,LAG_INFO
0,FOUR,AS,-11.0,21.0,1448,0,1,04:25:00,ANC--SEA
1,FOUR,AA,-8.0,12.0,2330,0,1,07:40:00,LAX--PBI
2,FOUR,US,-2.0,16.0,2296,0,0,07:46:00,SFO--CLT
3,FOUR,AA,-5.0,15.0,2342,0,1,07:45:00,LAX--MIA
4,FOUR,AS,-1.0,11.0,1448,0,1,02:55:00,SEA--ANC


- departure timeing AM/PM
- `SCHEDULED_DEPARTURE` : Planned Departure Time
- `DEPARTURE_TIME` : WHEEL_OFF - TAXI_OUT
- `DEPARTURE_DELAY` : Total Delay on Departure in Minutes
- `TAXI_OUT` : The time duration elapsed between departure from the origin airport gate and wheels off
- `WHEELS_OFF` : The time point that the aircraft's wheels leave the ground
- `SCHEDULED_TIME` : Planned time amount needed for the flight trip
- `AIR_TIME` : The time duration between wheels_off and wheels_on time
- `DISTANCE` : Distance between two airports

- `SCHEDULED_ARRIVAL` : Planned arrival time

- `ARRIVAL_DELAY` : ARRIVAL_TIME-SCHEDULED_ARRIVAL
- `DIVERTED` : Aircraft landed on airport that out of schedule

 /**convert time to decimal minutes
  rawtime = pd.to_datetime(t, format='%H:%M:%S').time() 
   print(t.hour, rawtime.minute)
  m = (hours * 60) + minutes + (seconds / 60). Ti
  times_in_decimal=(t.hour * 3600) + (t.minute * 60) 
def convert_hour_to_decimal_minute(rawtime):
    if pd.isna(rawtime) :
        np.nan
    else:
        print(type(rawtime),rawtime.total_seconds())
        h=rawtime.total_seconds()//3600
        print(h)        
        m=(rawtime.total_seconds()%3600)/60
        print(m)
        times_in_decimal = (h * 60) + m
        #     print(times_in_decimal)
        return times_in_decimal **/

In [39]:
dfTest=dfFlight.loc[:5,:] 
dfTest['DEPARTURE_TIME_DECIMAL']=dfTest['TOTAL_JOURNERY_TIME'].map(lambda dt : np.nan if dt is None  else (dt.total_seconds()//3600 * 60) + (dt.total_seconds()%3600)/60)
dfFlight.shape

C:\Users\S727953\AppData\Local\Continuum\anaconda3\envs\MLND_AI\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(469968, 9)

In [40]:
dfTest.head()

,DAY_OF_WEEK,AIRLINE,DEPARTURE_DELAY,TAXI_OUT,DISTANCE,DIVERTED,FLIGHT_DELAY,TOTAL_JOURNERY_TIME,LAG_INFO,DEPARTURE_TIME_DECIMAL
0,FOUR,AS,-11.0,21.0,1448,0,1,04:25:00,ANC--SEA,265.0
1,FOUR,AA,-8.0,12.0,2330,0,1,07:40:00,LAX--PBI,460.0
2,FOUR,US,-2.0,16.0,2296,0,0,07:46:00,SFO--CLT,466.0
3,FOUR,AA,-5.0,15.0,2342,0,1,07:45:00,LAX--MIA,465.0
4,FOUR,AS,-1.0,11.0,1448,0,1,02:55:00,SEA--ANC,175.0


In [41]:
# dfFlight = dfFlight.reindex(sorted(dfFlight.columns), axis=1)
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,DEPARTURE_DELAY,TAXI_OUT,DISTANCE,DIVERTED,FLIGHT_DELAY,TOTAL_JOURNERY_TIME,LAG_INFO
0,FOUR,AS,-11.0,21.0,1448,0,1,04:25:00,ANC--SEA
1,FOUR,AA,-8.0,12.0,2330,0,1,07:40:00,LAX--PBI
2,FOUR,US,-2.0,16.0,2296,0,0,07:46:00,SFO--CLT
3,FOUR,AA,-5.0,15.0,2342,0,1,07:45:00,LAX--MIA
4,FOUR,AS,-1.0,11.0,1448,0,1,02:55:00,SEA--ANC


In [42]:
dfFlight.dtypes

DAY_OF_WEEK                     object
AIRLINE                         object
DEPARTURE_DELAY                float64
TAXI_OUT                       float64
DISTANCE                         int64
DIVERTED                         int64
FLIGHT_DELAY                     int64
TOTAL_JOURNERY_TIME    timedelta64[ns]
LAG_INFO                        object
dtype: object

In [43]:
dfFlight['TOTAL_JOURNERY_TIME']=dfFlight['TOTAL_JOURNERY_TIME'].map(lambda dt : np.nan if dt is None  else (dt.total_seconds()//3600 * 60) + (dt.total_seconds()%3600)/60)

In [44]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,DEPARTURE_DELAY,TAXI_OUT,DISTANCE,DIVERTED,FLIGHT_DELAY,TOTAL_JOURNERY_TIME,LAG_INFO
0,FOUR,AS,-11.0,21.0,1448,0,1,265.0,ANC--SEA
1,FOUR,AA,-8.0,12.0,2330,0,1,460.0,LAX--PBI
2,FOUR,US,-2.0,16.0,2296,0,0,466.0,SFO--CLT
3,FOUR,AA,-5.0,15.0,2342,0,1,465.0,LAX--MIA
4,FOUR,AS,-1.0,11.0,1448,0,1,175.0,SEA--ANC


In [45]:
df=pd.get_dummies(dfFlight, drop_first=False)
df.columns

Index(['DEPARTURE_DELAY', 'TAXI_OUT', 'DISTANCE', 'DIVERTED', 'FLIGHT_DELAY',
       'TOTAL_JOURNERY_TIME', 'DAY_OF_WEEK_FIVE', 'DAY_OF_WEEK_FOUR',
       'DAY_OF_WEEK_ONE', 'DAY_OF_WEEK_SEVEN',
       ...
       'LAG_INFO_XNA--DEN', 'LAG_INFO_XNA--DFW', 'LAG_INFO_XNA--EWR',
       'LAG_INFO_XNA--IAH', 'LAG_INFO_XNA--LGA', 'LAG_INFO_XNA--MSP',
       'LAG_INFO_XNA--ORD', 'LAG_INFO_YAK--CDV', 'LAG_INFO_YAK--JNU',
       'LAG_INFO_YUM--PHX'],
      dtype='object', length=4197)

In [46]:
df.shape

(469968, 4197)

In [47]:
df.dropna(axis=0, inplace=True)
df.shape

(458092, 4197)

In [48]:
df.columns

Index(['DEPARTURE_DELAY', 'TAXI_OUT', 'DISTANCE', 'DIVERTED', 'FLIGHT_DELAY',
       'TOTAL_JOURNERY_TIME', 'DAY_OF_WEEK_FIVE', 'DAY_OF_WEEK_FOUR',
       'DAY_OF_WEEK_ONE', 'DAY_OF_WEEK_SEVEN',
       ...
       'LAG_INFO_XNA--DEN', 'LAG_INFO_XNA--DFW', 'LAG_INFO_XNA--EWR',
       'LAG_INFO_XNA--IAH', 'LAG_INFO_XNA--LGA', 'LAG_INFO_XNA--MSP',
       'LAG_INFO_XNA--ORD', 'LAG_INFO_YAK--CDV', 'LAG_INFO_YAK--JNU',
       'LAG_INFO_YUM--PHX'],
      dtype='object', length=4197)

In [49]:
Y=df['FLIGHT_DELAY']
df.drop('FLIGHT_DELAY',  axis=1, inplace=True) 

In [50]:
X=df

In [51]:
print(Y.shape )
print(X.shape)

(458092,)
(458092, 4196)


In [52]:
X.head()

,DEPARTURE_DELAY,TAXI_OUT,DISTANCE,DIVERTED,TOTAL_JOURNERY_TIME,DAY_OF_WEEK_FIVE,DAY_OF_WEEK_FOUR,DAY_OF_WEEK_ONE,DAY_OF_WEEK_SEVEN,DAY_OF_WEEK_SIX,...,LAG_INFO_XNA--DEN,LAG_INFO_XNA--DFW,LAG_INFO_XNA--EWR,LAG_INFO_XNA--IAH,LAG_INFO_XNA--LGA,LAG_INFO_XNA--MSP,LAG_INFO_XNA--ORD,LAG_INFO_YAK--CDV,LAG_INFO_YAK--JNU,LAG_INFO_YUM--PHX
0,-11.0,21.0,1448,0,265.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-8.0,12.0,2330,0,460.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-2.0,16.0,2296,0,466.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-5.0,15.0,2342,0,465.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-1.0,11.0,1448,0,175.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Normalization of data

In [53]:
normal=MinMaxScaler()
normal.fit_transform(X)

C:\Users\S727953\AppData\Local\Continuum\anaconda3\envs\MLND_AI\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [54]:
df=DecisionTreeClassifier()

In [55]:
x_train, x_test, y_train,y_test =train_test_split(X,Y, test_size=.15)

In [56]:
print(x_train.shape,y_train.shape)

(389378, 4196) (389378,)


In [57]:
df.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [58]:
y_pred=df.predict(x_test)

In [59]:
accuracy_score(y_pred, y_test)

0.8100823704048665

In [60]:
recall_score(y_pred, y_test)

0.8371137959027745

In [61]:
precision_score(y_pred, y_test)

0.84915714008913

In [62]:
roc_auc_score(y_pred, y_test)

0.802501441779652

In [63]:
roc_curve(y_pred, y_test)

(array([0.        , 0.23211091, 1.        ]),
 array([0.       , 0.8371138, 1.       ]),
 array([2, 1, 0], dtype=int64))

# Convert all hour to miniitutes and all minitus to decimal
----

# Model Training and Test

In [ ]:
dfAirline=ut.load_csv('././data/airlines.csv',numRow=300)

In [ ]:
airline_companies = dfAirline.set_index('IATA_CODE')['AIRLINE'].to_dict()

In [ ]:
airline_companies

In [ ]:
# dfFlight(['FLIGHT_DELAY','AIRLINE']).groupby(['AIRLINE']).agg('count')
dfGroup=dfFlight.groupby(['AIRLINE','FLIGHT_DELAY'])

In [ ]:
dfGroup=dfGroup['FLIGHT_DELAY'].agg(['count']).reset_index()

In [ ]:
dfGroup

In [ ]:
dfGroup
dfGroup.columns=['AIRLINE','FLIGHT_DELAY','DELAY_COUNT']

In [ ]:
dfGroup['DELAY_COUNT_%']=dfGroup['DELAY_COUNT'].map(missing_percent)

#### Delay Analysis in respect to Air line 

In [ ]:
dfGroup['AIRLINE']=dfGroup['AIRLINE'].map(airline_companies)

In [ ]:
dfGroup

In [ ]:
# function that extract statistical parameters from a grouby objet:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
#_______________________________________________________________
# Creation of a dataframe with statitical infos on each airline:
global_stats = dfFlight['DEPARTURE_DELAY'].groupby(dfFlight['AIRLINE']).apply(get_stats).unstack()
global_stats = global_stats.sort_values('count')
global_stats

In [ ]:
font = {'family' : 'normal', 'weight' : 'bold', 'size'   : 15}
mpl.rc('font', **font)

In [ ]:
#__________________________________________________________________
# I extract a subset of columns and redefine the airlines labeling 
df2 = dfFlight.loc[:, ['AIRLINE', 'DEPARTURE_DELAY']]
df2['AIRLINE'] = df2['AIRLINE'].replace(airline_companies)
#________________________________________________________________________
colors = ['royalblue', 'grey', 'wheat', 'c', 'firebrick', 'seagreen', 'lightskyblue',
          'lightcoral', 'yellowgreen', 'gold', 'tomato', 'violet', 'aquamarine', 'chartreuse']
#___________________________________
fig = plt.figure(1, figsize=(16,15))
gs=GridSpec(2,2)             
ax1=fig.add_subplot(gs[0,0]) 
ax2=fig.add_subplot(gs[0,1]) 
ax3=fig.add_subplot(gs[1,:]) 
#------------------------------
# Pie chart nº1: nb of flights
#------------------------------
labels = [s for s in  global_stats.index]
sizes  = global_stats['count'].values
explode = [0.3 if sizes[i] < 20000 else 0.0 for i in range(len(airline_companies))]
patches, texts, autotexts = ax1.pie(sizes, explode = explode,
                                labels=labels, colors = colors,  autopct='%1.0f%%',
                                shadow=False, startangle=0)
for i in range(len(airline_companies)): 
    texts[i].set_fontsize(14)
ax1.axis('equal')
ax1.set_title('% of flights per company', bbox={'facecolor':'midnightblue', 'pad':5},
              color = 'w',fontsize=18)
#_______________________________________________
# I set the legend: abreviation -> airline name
comp_handler = []
for i in range(len(airline_companies)):
    comp_handler.append(mpatches.Patch(color=colors[i],
            label = global_stats.index[i] + ': ' + airline_companies[global_stats.index[i]]))
ax1.legend(handles=comp_handler, bbox_to_anchor=(0.2, 0.9), 
           fontsize = 13, bbox_transform=plt.gcf().transFigure)
#----------------------------------------
# Pie chart nº2: mean delay at departure
#----------------------------------------
sizes  = global_stats['mean'].values
sizes  = [max(s,0) for s in sizes]
explode = [0.0 if sizes[i] < 20000 else 0.01 for i in range(len(airline_companies))]
patches, texts, autotexts = ax2.pie(sizes, explode = explode, labels = labels,
                                colors = colors, shadow=False, startangle=0,
                                autopct = lambda p :  '{:.0f}'.format(p * sum(sizes) / 100))
for i in range(len(airline_companies)): 
    texts[i].set_fontsize(14)
ax2.axis('equal')
ax2.set_title('Mean delay at origin', bbox={'facecolor':'midnightblue', 'pad':5},
              color='w', fontsize=18)
#------------------------------------------------------
# striplot with all the values reported for the delays
#___________________________________________________________________
# I redefine the colors for correspondance with the pie charts
colors = ['firebrick', 'gold', 'lightcoral', 'aquamarine', 'c', 'yellowgreen', 'grey',
          'seagreen', 'tomato', 'violet', 'wheat', 'chartreuse', 'lightskyblue', 'royalblue']
#___________________________________________________________________
ax3 = sns.stripplot(y="AIRLINE", x="DEPARTURE_DELAY", size = 4, palette = colors,
                    data=df2, linewidth = 0.5,  jitter=True)
plt.setp(ax3.get_xticklabels(), fontsize=14)
plt.setp(ax3.get_yticklabels(), fontsize=14)
ax3.set_xticklabels(['{:2.0f}h{:2.0f}m'.format(*[int(y) for y in divmod(x,60)])
                         for x in ax3.get_xticks()])
plt.xlabel('Departure delay', fontsize=18, bbox={'facecolor':'midnightblue', 'pad':5},
           color='w', labelpad=20)
ax3.yaxis.label.set_visible(False)
#________________________
plt.tight_layout(w_pad=3) 